In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

import utils


In [ ]:
import os
data_folder_path = "./data"
print(os.listdir(data_folder_path))

x_train, y_train, x_test = utils.load_data(data_folder_path)

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

@ignore_warnings(category=ConvergenceWarning)
def cv_rf(X_train, y_train, X_valid, y_valid, h: dict):
    c = LogisticRegression(penalty='elasticnet', C=h.get('lambda', 1), l1_ratio=h.get('alpha', None), \
                           solver='saga', \
                           random_state = 54321, n_jobs = -1) # set n_jobs to speed up by using more CPU cores)
    scaler = StandardScaler().fit(X_train) 
    m = c.fit(scaler.transform(X_train), y_train)
    p = m.predict(scaler.transform(X_valid))

    return utils.evaluate(y_valid, p)

In [31]:
hyperparameters = {"lambda_inverse": (10**i for i in range(-5, 6)),
                   "alpha": np.arange(0, 1.1, .1)}

In [24]:

pr = utils.kfold_cv(x_train, y_train, k = 10, H = list(utils.create_H(hyperparameters)), cv_fun = cv_rf, random_state = 12345)
pr

,accuracy,precision,recall,f,_h
0,0.621547,0.310774,0.500000,0.383305,"{'lambda': 1e-05, 'alpha': 0.0}"
1,0.621547,0.310774,0.500000,0.383305,"{'lambda': 1e-05, 'alpha': 0.1}"
2,0.621547,0.310774,0.500000,0.383305,"{'lambda': 1e-05, 'alpha': 0.2}"
3,0.621547,0.310774,0.500000,0.383305,"{'lambda': 1e-05, 'alpha': 0.30000000000000004}"
4,0.621547,0.310774,0.500000,0.383305,"{'lambda': 1e-05, 'alpha': 0.4}"
...,...,...,...,...,...
116,0.774970,0.763364,0.750382,0.755190,"{'lambda': 100000, 'alpha': 0.6000000000000001}"
117,0.774970,0.763364,0.750382,0.755190,"{'lambda': 100000, 'alpha': 0.7000000000000001}"
118,0.774970,0.763364,0.750382,0.755190,"{'lambda': 100000, 'alpha': 0.8}"
119,0.774970,0.763364,0.750382,0.755190,"{'lambda': 100000, 'alpha': 0.9}"


In [29]:
best_model = utils.get_best(pr)
best_model

0.77915931875416
{'lambda': 0.01, 'alpha': 0.7000000000000001}


accuracy                                          0.779159
precision                                         0.770796
recall                                            0.749827
f                                                 0.756758
_h           {'lambda': 0.01, 'alpha': 0.7000000000000001}
Name: 40, dtype: object